<center><img style="width:500px;" src="https://s3.amazonaws.com/thinkific-import/370184/U9zyVFboQScufA7Iih3p_Capture_d_e_cran_2020_09_23_a__00_11_29_png"></center>

# Parcours : Le Data Mining pour tous

## Niveau 3 : Automatiser votre travail

### Cas pratique : Les données de marché

#### Etape 0 : Initialisation du notebook

In [1]:
# Import des modules

import pandas as pd
from datetime import datetime, timedelta, date

#### Etape 1 : Récupération des données sources

<b> Source Yahoo Finance : </b> <br> 
https://finance.yahoo.com/ <br>

In [2]:
# Récupérer une donnée sur yahoo finance
url = 'https://query1.finance.yahoo.com/v7/finance/download/^GSPC?period1=1569172118&period2=1600794518&interval=1d&events=history'
df = pd.read_csv(url)
df.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-09-23,2983.500000,2999.149902,2982.229980,2991.780029,2991.780029,3186590000
1,2019-09-24,3002.429932,3007.979980,2957.729980,2966.600098,2966.600098,3868160000
2,2019-09-25,2968.350098,2989.820068,2952.860107,2984.870117,2984.870117,3318870000
3,2019-09-26,2985.729980,2987.280029,2963.709961,2977.620117,2977.620117,3077240000
4,2019-09-27,2985.469971,2987.310059,2945.530029,2961.790039,2961.790039,3243650000


In [3]:
# Création d'une table avec toute la liste des tickers que je souhaites récupérer
ref_tickers = pd.read_excel('REF_MARKET-DATA.xlsx',
                            sheet_name='TICKERS')

ref_tickers.head(5)

,TYPOLOGY,TICKER,LABEL,ENTITIES
0,INDICE,^GSPC,S&P 500,AMERICAS
1,INDICE,^DJI,Dow,AMERICAS
2,INDICE,^IXIC,NASDAQ,AMERICAS
3,INDICE,^NO,MXSE,AMERICAS
4,INDICE,^BVSP,Ibovespa,AMERICAS


In [4]:
# Variabilisation des dates (bornes)
period_deb = str(int(datetime(2020, 1, 1).timestamp()))
period_fin = str(int(datetime.now().timestamp()))

In [5]:
# Initialisation du dataframe
data_tickers = pd.DataFrame()

# Récupération de tous les tickers et création de la base de donnée tickers
tickers = ref_tickers['TICKER'].drop_duplicates()

for ticker in tickers:
    # Récupération des données
    tmp_df = pd.DataFrame()
    try:
        url = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={period_deb}&period2={period_fin}&interval=1d&events=history'
        tmp_df = pd.read_csv(url).reset_index(drop=True)
    except:
        print(f'Url not working : {ticker}')
        
    if len(tmp_df) > 1:       
        # Enrichissement
        tmp_df['TICKER'] = ticker
        tmp_df = tmp_df[['Date', 'TICKER', 'Close']]
        data_tickers = pd.concat([data_tickers, tmp_df], axis=0)

data_tickers.head(5)

Url not working : ^NO
Url not working : ^E3X.FGI
Url not working : ^RTS.RS


,Date,TICKER,Close
0,2019-12-31,^GSPC,3230.780029
1,2020-01-02,^GSPC,3257.850098
2,2020-01-03,^GSPC,3234.850098
3,2020-01-06,^GSPC,3246.280029
4,2020-01-07,^GSPC,3237.179932


#### Etape 2 - Nettoyage de la base

In [6]:
# Initialisation du dataframe
data_clean = data_tickers.copy()

# Mise au format de la date
data_clean['Date'] = pd.to_datetime(data_clean['Date'], format='%Y-%m-%d')

# Suppression des valeurs nulles dans Close
data_clean = data_clean.dropna(subset=['Close'])

# Renomage de la colonne 'Close' en 'VALUE'
data_clean = data_clean.rename(columns={'Close': 'VALUE'})

# Format des colonnes en majuscule
data_clean.columns = map(lambda x: str(x).upper(), data_clean.columns)
    
data_clean.head(5)

,DATE,TICKER,VALUE
0,2019-12-31,^GSPC,3230.780029
1,2020-01-02,^GSPC,3257.850098
2,2020-01-03,^GSPC,3234.850098
3,2020-01-06,^GSPC,3246.280029
4,2020-01-07,^GSPC,3237.179932


#### Etape 3 - Enrichissement des données

3.1 - CALCUL DE LA VALEUR DU J-1

In [7]:
# Initialisation du dataframe
data_enr = data_clean.copy()

# Ajout des indexes dans le data frame principal
data_value = data_enr.copy()
data_value['INDEX_D-1'] = data_value.index - 1

# Création du dataframe pour récupérer la valeur précédente
data_value_last = data_enr.copy()
data_value_last['INDEX_D-1'] = data_value_last.index 
data_value_last = data_value_last.rename(columns={'VALUE': 'VALUE_D-1'})
data_value_last = data_value_last.drop(['DATE'], axis=1)

data_bdd = pd.merge(data_value,
                    data_value_last,
                    on=['TICKER', 'INDEX_D-1'],
                    how='left')

data_bdd = data_bdd.drop(['INDEX_D-1'], axis=1)
data_bdd.head(5)

,DATE,TICKER,VALUE,VALUE_D-1
0,2019-12-31,^GSPC,3230.780029,NaN
1,2020-01-02,^GSPC,3257.850098,3230.780029
2,2020-01-03,^GSPC,3234.850098,3257.850098
3,2020-01-06,^GSPC,3246.280029,3234.850098
4,2020-01-07,^GSPC,3237.179932,3246.280029


3.2 - ENRICHISSEMENT DES DONNES TICKERS

In [8]:
# Ajout des informations supplémentaires
data_bdd = pd.merge(data_bdd, 
                    ref_tickers, 
                    on=['TICKER'],
                    how='left'
                   )

data_bdd.head(5)

,DATE,TICKER,VALUE,VALUE_D-1,TYPOLOGY,LABEL,ENTITIES
0,2019-12-31,^GSPC,3230.780029,NaN,INDICE,S&P 500,AMERICAS
1,2020-01-02,^GSPC,3257.850098,3230.780029,INDICE,S&P 500,AMERICAS
2,2020-01-03,^GSPC,3234.850098,3257.850098,INDICE,S&P 500,AMERICAS
3,2020-01-06,^GSPC,3246.280029,3234.850098,INDICE,S&P 500,AMERICAS
4,2020-01-07,^GSPC,3237.179932,3246.280029,INDICE,S&P 500,AMERICAS


3.3 - CREATION DES DONNES "GLOBAL"

In [9]:
data_global = data_bdd.copy()

# Liste des tickers à selectionner
tickers_list = ['^GSPC','^DJI','^FCHI','^FTSE','^HSI']

# Filtre des valeurs
data_global = data_global[data_global['TICKER'].isin(tickers_list)]

# Modification de la colonne ENTITIES
data_global['ENTITIES'] = 'GLOBAL'

# Concaténation des 2 dataframes
data_bdd = pd.concat([data_bdd, data_global], axis=0)
data_bdd.head(5)

,DATE,TICKER,VALUE,VALUE_D-1,TYPOLOGY,LABEL,ENTITIES
0,2019-12-31,^GSPC,3230.780029,NaN,INDICE,S&P 500,AMERICAS
1,2020-01-02,^GSPC,3257.850098,3230.780029,INDICE,S&P 500,AMERICAS
2,2020-01-03,^GSPC,3234.850098,3257.850098,INDICE,S&P 500,AMERICAS
3,2020-01-06,^GSPC,3246.280029,3234.850098,INDICE,S&P 500,AMERICAS
4,2020-01-07,^GSPC,3237.179932,3246.280029,INDICE,S&P 500,AMERICAS


#### Etape 4 : Calcul

In [12]:
data_bdd['VARV'] = data_bdd['VALUE'] - data_bdd['VALUE_D-1']
data_bdd['VARP'] = (data_bdd['VARV'] / abs(data_bdd['VALUE_D-1']))

#### Etape 5 : Sauvegarde et exposition du csv

In [11]:
# Ajout de la date de mise à jour
data_bdd['DATE_MAJ'] = datetime.now().strftime('%Y/%m/%d %H:%M:%S')

# Sauvegarde des données au format csv
data_bdd.to_csv('BASE_MARKET-DATA.csv',
                sep=";",
                decimal=',',
                index=False)

print('CSV sauvegardé dans le répertoire courant.')

CSV sauvegardé dans le répertoire courant.
